In [2]:
import os
import sys
import matplotlib.pyplot as plt

os.chdir('..')

import numpy as np
import torch
import pandas as pd
import random

from models.TrainPlanningOperator3D import PlanningOperator3D

from planners import generaterandompos, getPNOVal, getFMMVal
from planners import AStarPlanner, FMMPlanner, PlanningOperatorPlanner, testplanneronmaps, plot_2d_map_and_two_paths



Load Gibson Environment

In [4]:
Ntotal = 800

ntest = 160
ntrain = 640

sub = 1
Sx = 62
Sy = 160
Sz = 160

maps = np.load('/mountvol/igib-dataset-160-5G/mask.npy')
maps = maps[:Ntotal][-ntest:, ::sub, ::sub, ::sub][:, :Sx, :Sy, :Sz]
# maps = maps[:Ntotal][:ntrain:-, ::sub, ::sub, ::sub][:, :Sx, :Sy, :Sz]
# maps = maps[0:8,:,:,:]
# maps = np.repeat(maps, repeats=5, axis=0)



Load Model

In [3]:


# Model
modes = 8
width = 12
nlayers = 1

model = PlanningOperator3D(modes, modes, modes, width, nlayers)
model.load_state_dict(torch.load("/mountvol/igib-dataset-160-5G/planningoperator_env160_m8_w12_l1_b5_lr3e-3_5g_26sep/n320_lr3.000000e-03_gamma6.000000e-01_wd3.000000e-06_seed5/model3d.ckpt"))
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


/tmp/ipykernel_16804/19159190.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/mountvol/igibchunk-dataset-84-10g/planningoperator_env84

PlanningOperator3D(
  (fc0): Linear(in_features=3, out_features=32, bias=True)
  (conv0): SpectralConv3d()
  (w0): Conv3d(32, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (fc1): DeepNormMetric(
    (Us): ModuleList(
      (0-1): 2 x Linear(in_features=32, out_features=128, bias=False)
    )
    (Ws): ModuleList(
      (0): ConstrainedLinear(in_features=128, out_features=128, bias=False)
    )
    (activation): MaxReLUPairwiseActivation(
      (avg_pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
    (output_activation): ConcaveActivation()
    (reduce_metric): ReduceMetric()
  )
)

Generate Random Positions

In [5]:
import numpy as np

# Assuming mask is of shape (100, X, Y, Z) and goalpositions is generated for 100 samples
samuelsmask = mask[0,:,:,:]
samuelsmask_array = np.repeat(samuelsmask[np.newaxis, :, :, :], 100, axis=0)

# Generate random positions (assumed to return a 100x3 array for 100 goal positions)
goalpositions = generaterandompos(samuelsmask_array)

val_PNO_array = []

# Loop through the 100 masks and goal positions
for i in range(100):
    val_PNO, _ = getPNOVal(goalpositions[i, :], boltonmask_array[i], model)
    valFMM,_ = getFMMVal(goalpositions[i, :], boltonmask_array[i])
    val_PNO_array.append(val_PNO)

# Convert the list to a NumPy array for easier manipulation
val_PNO_array = np.array(val_PNO_array)
